In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #생성되는 이미지를 관찰하기 위함입니다. 3차원 플롯을 그리는 용도입니다.
from matplotlib import cm # 데이터포인트에 색상을 입히는 것에 사용됩니다.
import numpy as np

In [ ]:
EPOCH = 10
BATCH_SIZE = 64
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("using Device: ", DEVICE)

### Fashion MNIST 데이터셋을 사용해서 Autoencoder코드 구현 따라가기

In [ ]:
# trainset을 불러오는 코드 (datasets 안에 있는 예시 데이터셋)
trainset = datasets.FashionMNIST(
    root= './.data/',
    train = True,
    download = True,
    transform = transforms.ToTensor()
)

In [ ]:
# dataloader 짜기 (데이터를 불러오는 과정과 초기 세팅해주기)

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

# shuffle : 전체 학습데이터를 배열 인덱스와 관계없이 만드는 작업을 말함
# 만일 안하게 되면 신경망이 해당 순서까지도 예측하기 때문에 과적합 발생할 수 있음 따라서 shuffle진행해줘야함

# num_workers : default 값은 0임 how many subprocesses to use for data loading.  data 로딩을 위해 몇 개의 서브 프로세스를 사용할 것인지를 결정


In [ ]:
# 오토인코더 모듈 정의

class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder,self).__init__()

  # encoder정의하기 간단한 신경망으로 분류모델 처럼 생김
    self.encoder = nn.Sequential(
        # nn.Sequential을 사용해서 encoder와 decoder 두 모듈을 순차적으로 엮엊무
        nn.Linear(28*28,128), #차원을 28*28에서 점차 줄여나가기
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,12),
        nn.ReLU(),
        nn.Linear(12,3), #입력의 특징을 3차원으로 압축하기 => 출력값이 바로 잠재변수 할당
    )
    self.decoder = nn.Sequential(
        nn.Linear(3,12), #디코더는 차원을 점차 28*28로 복원하기ㅏ
        nn.ReLU(),
        nn.Linear(12,64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128,28*28),
        nn.Sigmoid(),
        # sigmoid 마지막에 0-1사이로 출력하는 함수 추가
    )

  def forward(self,x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return encoded, decoded

In [ ]:
autoencoder = Autoencoder().to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.005)
# Adam을 최적화 함수로 사용

criterion = nn.MSELoss()
# 손실함수로는 MSELoss사용 / 인코더와 디코더에서 나온 값의 차이를 계산하기 위해 평균제곱오차 함수 사용


In [ ]:
# 원본 이미지 시각화 하기

view_data = trainset.data[:5].view(-1, 28*28)
view_data = view_data.type(torch.FloatTensor)/255.

In [ ]:
# 학습하기 위한 함수
def train(autoencoder, train_loader):
    autoencoder.train()
    for step, (x, label) in enumerate(train_loader):
        x = x.view(-1, 28*28).to(DEVICE)
        y = x.view(-1, 28*28).to(DEVICE) #x(입력)와 y(대상 레이블)모두 원본이미지(x)인 것을 주의해야 합니다.
        label = label.to(DEVICE)
        # 해당 train_loader의 스텝마다 x와 label값을 같이 가져오기 enumerate 사용

        encoded, decoded = autoencoder(x)

        loss = criterion(decoded, y) # decoded와 원본이미지(y) 사이의 평균제곱오차를 구합니다
        optimizer.zero_grad() #기울기에 대한 정보를 초기화합니다.
        loss.backward() # 기울기를 구합니다.
        optimizer.step() #최적화를 진행합니다.

In [ ]:
# 학습시작

for epoch in range(1,EPOCH+1):
  train(autoencoder, train_loader)

  text_x = view_data.to(DEVICE)
  _, decoded_data = autoencoder(text_x)

  f,a = plt.subplots(2,5,figsize=(5,2))
  print("[Epoch{}]".format(epoch))
  for i in range(5):
    img = np.reshape(view_data.data.numpy()[i],(28,28))
    a[0][i].imshow(img, cmap="gray")
    a[0][i].set_xticks(()); a[0][i].set_yticks(())

  for i in range(5):
    img = np.reshape(decoded_data.to("cpu").data.numpy()[i], (28,28))
    a[1][i].imshow(img,cmap='gray')
    a[1][i].set_xticks(()); a[1][i].set_yticks(())
  plt.show()

In [ ]:
# 잠재변수를 3D 플롯으로 시각화하기
view_data = trainset.data[:200].view(-1,28*28) #원본 이미지 200개 준비하기
view_data = view_data.type(torch.FloatTensor)/255.

test_x = view_data.to(DEVICE)
encoded_data, _ = autoencoder(test_x)
encoded_data = encoded_data.to('cpu')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

CLASSES = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

%matplotlib inline

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

X = encoded_data.data[:, 0].numpy()
Y = encoded_data.data[:, 1].numpy()
Z = encoded_data.data[:, 2].numpy()

labels = trainset.targets[:200].numpy()

for x, y, z, s in zip(X, Y, Z, labels):
    name = CLASSES[s]
    color = cm.rainbow(int(255 * s / 9))
    ax.text(x, y, z, name, backgroundcolor=color)

ax.set_xlim(X.min(), X.max())
ax.set_ylim(Y.min(), Y.max())
ax.set_zlim(Z.min(), Z.max())

plt.show()


3d 상에 해당 LABEL이 잘 분류되어 있는지 분포를 시각화해봐도 !

같은 레이블을 가진 이미지의 잠재변수는 같이 모여있음을 확인해볼 수 있었음 !!!

### 오토인코더로 망가진 이미지 복원하기

In [ ]:
# 먼저 노이즈를 더하는 과정

def add_noise(img):
  noise = torch.randn(img.size()) *0.2
  noisy_img = img + noise
  return noisy_img


In [ ]:
# 학습 코드에 노이즈 더하는 코드 추가하기

def train(autoencoder,train_loader):
  autoencoder.train()
  avg_loss = 0

  for step,(x,label) in enumerate(train_loader):
     noisy_x = add_noise(x)
     noisy_x = noisy_x.view(-1,28*28).to(DEVICE)
     y = x.view(-1,28*28).to(DEVICE)

     label = label.to(DEVICE)
     encoded, decoded = autoencoder(noisy_x)

     loss = criterion(decoded,y)
     optimizer.zero_grad()
     loss.backward()
     optimizer.step()

     avg_loss +=loss.item()
    #  평균 오찻값 확인하기
  return avg_loss / len(train_loader)


In [ ]:
for epoch in range(1,EPOCH+1):
  loss = train(autoencoder,train_loader)
  print("[EPOCH {}] loss: {}".format(epoch, loss))

In [ ]:
# 잡음제거 시각화

testset = datasets.FashionMNIST(
    root="./.data/",
    train = False,
    download = True,
    transform = transforms.ToTensor()
)

# 테스트셋에서 이미지 한장을 가져오기
sample_data = testset.data[0].view(-1,28*28)
sample_data = sample_data.type(torch.FloatTensor)/255.

original_x = sample_data[0]
noisy_x = add_noise(original_x).to(DEVICE)
_, recovered_x = autoencoder(noisy_x)

In [ ]:
f,a = plt.subplots(1,3,figsize=(15,15))

original_img = np.reshape(original_x.to('cpu').data.numpy(), (28,28))
noisy_img = np.reshape(noisy_x.to('cpu').data.numpy(), (28,28))
recovered_img = np.reshape(recovered_x.to('cpu').data.numpy(), (28,28))

a[0].set_title("Original")
a[0].imshow(original_img, cmap="gray")

a[1].set_title('noisy_img')
a[1].imshow(noisy_img, cmap='gray')


a[2].set_title('Recovered')
a[2].imshow(recovered_img, cmap='gray')

plt.show()